<a href="https://colab.research.google.com/github/SergioManuelJob/RNN-Model/blob/master/7RO_RNN_Sergio_Manuel_Su%C3%A1rez_Su%C3%A1rez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Laboratorio: Modelos del lenguaje con RNNs

En este laboratorio, vamos a entrenar un modelo del lenguaje basado en caracteres con Recurrent Neural Networks. Asimismo, utilizaremos el modelo para generar texto. En particular, alimentaremos nuestro modelo con obras de la literatura clásica en castellano para obtener una red neuronal que sea capaz de "escribir" fragmentos literarios.

Los entrenamientos en esta laboratorio para obtener un modelo de calidad podrían tomar cierto tiempo (5-10 minutos por epoch), por lo que se aconseja empezar a trabajar pronto. El uso de GPUs no ayuda tanto con LSTMs como con CNNs, por lo que si tenéis máquinas potentes en casa es posible que podáis entrenar más rápido o a la misma velocidad que en Colab. En todo caso, la potencia de Colab es más que suficiente para completar este laboratorio con éxito.

<center><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/cb/Portada_Trafalgar_%281873%29.jpg/800px-Portada_Trafalgar_%281873%29.jpg" style="text-align: center" height="300px"></center>

El dataset a utilizar consistirá en un archivo de texto con el contenido íntegro en castellano de Trafalgar, disponible de manera libre en la página de [Project Gutenberg](https://www.gutenberg.org). Asimismo, como apartado optativo en este laboratorio se pueden utilizar otras fuentes de texto. Aquí podéis descargar los datos a utilizar de El Quijote y un par de obras adicionales:

[El ingenioso hidalgo Don Quijote de la Mancha (Miguel de Cervantes)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219424&authkey=AH0gb-qSo5Xd7Io)

[Compilación de obras teatrales (Calderón de la Barca)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219433&authkey=AKvGD6DC3IRBqmc)

[Trafalgar (Benito Pérez Galdós)](https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ)

Como ya deberíamos de estar acostumbrados en problemas de Machine Learning, es importante echar un vistazo a los datos antes de empezar.

## 1. Carga y procesado del texto

Primero, vamos a descargar el libro e inspeccionar los datos. El fichero a descargar es una versión en .txt del libro Trafalgar, a la cual se le han borrado introducciones, licencias y otras secciones para dejarlo con el contenido real de la novela.

In [1]:
import numpy as np
import keras
import matplotlib.pyplot as plt
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import random
import io

path = keras.utils.get_file(
fname="trafalgar.txt",
origin="https://onedrive.live.com/download?cid=C506CF0A4F373B0F&resid=C506CF0A4F373B0F%219434&authkey=AErPCAtMKOI5tYQ"
)

312774/312774 [==============================] - 1s 2us/step


Una vez descargado, vamos a leer el contenido del fichero en una variable. Adicionalmente, convertiremos el contenido del texto a minúsculas para ponérselo un poco más fácil a nuestro modelo (de modo que todas las letras sean minúsculas y el modelo no necesite diferenciar entre minúsculas y mayúsculas).

**1.1.** Leer todo el contenido del fichero en una única variable ***text*** y convertir el string a minúsculas

In [2]:
text=open(path).read().lower()

Podemos comprobar ahora que efectivamente nuestra variable contiene el resultado deseado, con el comienzo tan característico del Quijote.

In [3]:
print("Longitud del texto: {}".format(len(text)))
print(text[0:300])

Longitud del texto: 300039
-i-

se me permitirá que antes de referir el gran suceso de que fui testigo,
diga algunas palabras sobre mi infancia, explicando por qué extraña
manera me llevaron los azares de la vida a presenciar la terrible
catástrofe de nuestra marina.

al hablar de mi nacimiento, no imitaré a la mayor parte de


## 2. Procesado de los datos

Una de las grandes ventajas de trabajar con modelos que utilizan caracteres en vez de palabras es que no necesitamos tokenizar el texto (partirlo palabra a palabra). Nuestro modelo funcionará directamente con los caracteres en el texto, incluyendo espacios, saltos de línea, etc.

Antes de hacer nada, necesitamos procesar el texto en entradas y salidas compatibles con nuestro modelo. Como sabemos, un modelo del lenguaje con RNNs acepta una serie de caracteres y predice el siguiente carácter en la secuencia.

* "*Se me permitirá que antes de referir el gr*" -> predicción: **a**
* "*e me permitirá que antes de referir el gra*" -> predicción: **n**

De modo que la entrada y la salida de nuestro modelo necesita ser algo parecido a este esquema. En este punto, podríamos usar dos formas de preparar los datos para nuestro modelo.

1. **Secuencia a secuencia**. La entrada de nuestro modelo sería una secuencia y la salida sería esa secuencia trasladada un caracter a la derecha, de modo que en cada instante de tiempo la RNN tiene que predecir el carácter siguiente. Por ejemplo:

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: e me permitirá que antes de referir el gra

2. **Secuencia a carácter**. En este variante, pasaríamos una secuencia de caracteres por nuestra RNN y, al llegar al final de la secuencia, predeciríamos el siguiente carácter.

>* *Input*:  Se me permitirá que antes de referir el gr
>* *Output*: a

En este laboratorio, por simplicidad, vamos a utilizar la segunda variante (PERO USANDO PALABRAS NO CARACTERES).

De este modo, a partir del texto, hemos de generar nuestro propio training data que consista en secuencias de PALABRAS con la siguiente PALABRA a predecir. Para estandarizar las cosas, utilizaremos secuencias de tamaño *SEQ_LENGTH* PALABRAS (un hiperparámetro que podemos elegir nosotros).



#### 2.1. Obtención de las palabras y mapas de palabras

Antes que nada, necesitamos saber qué palabras aparecen en el texto, ya que tendremos que diferenciarlos mediante un índice de 0 a *num_words* - 1 en el modelo. Obtener:


1.   Número de caracteres únicos que aparecen en el texto.
2.   Diccionario que asocia char a índice único entre 0 y *num_words* - 1. Por ejemplo, {'se': 0, 'me': 1, ...}
3.   Diccionario reverso de índices a palabras: {0: 'se', 1: 'me', ...}


In [4]:
texto=""
for letra in text:
  if not letra in "?¿.,'¡!()-{};[]»«:\"":
    texto+=letra
  text=texto

palabras = text.split()
palabras_unicas = set(palabras)
word_index = {}
k=0
for palabra in palabras_unicas:
  word_index[palabra] = k
  k+=1

Ejecutar una sola vez

In [5]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

In [7]:
len(palabras)

51221

#### 2.2. Obtención de secuencias de entrada y palabra a predecir

Ahora, vamos a obtener las secuencias de entrada en formato texto y las correspondientes palabras a predecir. Para ello, recorrer el texto completo leído anteriormente, obteniendo una secuencia de SEQ_LENGTH palabras y la siguiente palabra a predecir. Una vez hecho, desplazarse una palabra a la derecha y hacer lo mismo para obtener una nueva secuencia y predicción. Guardar las secuencias en una variable ***sequences*** y las palabras a predecir en una variable ***next_words***.

Por ejemplo, si el texto fuera "Don Quijote" y SEQ_LENGTH fuese 2, tendríamos

* *sequences* = ["Don Quijote", "Quijote de"]
* *next_chars* = ['de', 'La']

In [8]:
SEQ_LENGTH = 5
step=1 # Para que la siguiente sentencia esté desplazada una palabra la izquierda.
sequences = []
next_words = []

Indicar el tamaño del training set que acabamos de generar.

In [9]:
for i in range(0,len(palabras)-SEQ_LENGTH, step):
  sequences.append(palabras[i:i+SEQ_LENGTH])
  next_words.append(palabras[i+SEQ_LENGTH])


In [10]:
sequences[8]

['el', 'gran', 'suceso', 'de', 'que']

In [11]:
next_words[8]

'fui'

Como el Quijote es muy largo y tenemos muchas secuencias, podríamos encontrar problemas de memoria. Por ello, vamos a elegir un número máximo de ellas. Si estás corriendo esto localmente y tienes problemas de memoria, puedes reducir el tamaño aún más, pero ten cuidado porque, a menos datos, peor calidad del modelo.

In [12]:
MAX_SEQUENCES = 376493

In [13]:
perm = np.random.permutation(len(sequences))
sequences, next_words = np.array(sequences), np.array(next_words)
sequences, next_words = sequences[perm], next_words[perm]
sequences, next_words = list(sequences[:MAX_SEQUENCES]), list(next_words[:MAX_SEQUENCES])

print(len(sequences))

51216


In [14]:
sequences[len(sequences)-1]

array(['oí', 'hablar', 'de', 'cierta', 'academia'], dtype='<U28')

In [15]:
next_words[len(sequences)-1]

'en'

In [16]:
frases = []
for i in range(len(sequences)):
  frase = ' '.join(sequences[i])
  frases.append(frase)
  #Tokenizar las frases
newsVec = [nltk.word_tokenize(frase) for frase in frases]

In [17]:
newsVec

[['sí', 'mismo', 'el', 'retumbante', 'título'],
 ['sobre', 'el', 'famoso', 'proyecto', 'y'],
 ['contramaestre', 'marinero', 'artillero', 'carpintero', 'y'],
 ['causaba', 'la', 'vista', 'de', 'los'],
 ['terrible', 'que', 'el', 'anterior', 'nos'],
 ['con', 'una', 'pierna', 'de', 'palo'],
 ['haber', 'oído', 'un', 'enojoso', 'sermón'],
 ['ha', 'quedado', 'relegada', 'al', 'olvido'],
 ['precioso', 'en', 'hacerse', 'el', 'coleto'],
 ['poca', 'gravedad', 'y', 'aunque', 'una'],
 ['de', 'diferentes', 'clases', 'con', 'la'],
 ['tú', 'irás', 'a', 'cádiz', 'tambiéndijo'],
 ['a', 'él', 'cuántas', 'veces', 'sentía'],
 ['penas', 'la', 'vergüenza', 'de', 'la'],
 ['escuadra', 'un', 'hombre', 'más', 'apto'],
 ['estado', 'de', 'mi', 'vida', 'y'],
 ['dios', 'una', 'cosa', 'y', 'tú'],
 ['el', 'navío', '_santa', 'ana_', 'se'],
 ['a', 'un', 'navío', 'yo', 'le'],
 ['dignidad', 'de', 'la', 'corona', 'pues'],
 ['desventuras', 'al', 'año', 'de', 'su'],
 ['anoche', 'y', 'esta', 'mañana', 'hablándote'],
 ['guardia

In [18]:
from gensim.models import Word2Vec
model=Word2Vec(newsVec, min_count=1, vector_size=32)
model.wv["caballero"]

array([-0.09281999, -0.4495089 ,  0.37317616,  0.27847937, -0.08410522,
       -0.3436988 ,  0.40628433,  0.24880463, -0.12750521,  0.03826626,
        0.17038073, -0.3255076 ,  0.02416263, -0.17249124, -0.2457167 ,
       -0.02233291, -0.13297512,  0.22310191,  0.08325858,  0.34731364,
        0.41419664,  0.5042874 ,  0.65491664, -0.33356512,  0.08339622,
        0.02619052, -0.3091404 ,  0.09547292,  0.02042916, -0.44525245,
       -0.2712552 , -0.01884866], dtype=float32)

In [19]:
model.wv.most_similar("caballero")

[('cuyo', 0.9964745044708252),
 ('apenas', 0.9961650967597961),
 ('esfuerzo', 0.9961567521095276),
 ('tener', 0.9958619475364685),
 ('varias', 0.9957494139671326),
 ('mirada', 0.9956908226013184),
 ('alguno', 0.9956010580062866),
 ('tono', 0.9954982399940491),
 ('terror', 0.9954501390457153),
 ('habiendo', 0.9949938058853149)]

#### 2.3. Obtención de input X y output y para el modelo

Finalmente, a partir de los datos de entrenamiento que hemos generado vamos a crear los arrays de datos X e y que pasaremos a nuestro modelo.

Para ello, vamos a utilizar *one-hot encoding* para nuestras palabras. Por ejemplo, si sólo tuviéramos 4 palabras (a, b, c, d), las representaciones serían: (1, 0, 0, 0), (0, 1, 0, 0), (0, 0, 1, 0) y (0, 0, 0, 1).

De este modo, **X** tendrá shape *(num_sequences, seq_length, num_words)* e **y** tendrá shape *(num_sequences, num_words)*.



In [20]:
NUM_SEQUENCES = len(sequences)
X = np.zeros((NUM_SEQUENCES, SEQ_LENGTH, 32))
y = np.zeros((NUM_SEQUENCES, len(palabras_unicas)))

In [21]:
for k in range(NUM_SEQUENCES):
  palabra=next_words[k]
  y[k,word_index[palabra]]=1 #codificación ONE-HOT

In [22]:
for k in range(NUM_SEQUENCES):
    for w in range(SEQ_LENGTH):
        word = sequences[k][w]
        if word in model.wv:
            X[k,w] = model.wv[word]
        else:
            pass

## 3. Definición del modelo y entrenamiento

Una vez tenemos ya todo preparado, es hora de definir el modelo. Define un modelo que utilice una **LSTM** con **128 unidades internas**. Si bien el modelo puede definirse de una manera más compleja, para empezar debería bastar con una LSTM más una capa Dense con el *softmax* que predice el siguiente caracter a producir. Adam puede ser una buena elección de optimizador.

Una vez el modelo esté definido, entrénalo un poco para asegurarte de que la loss es decreciente. No es necesario guardar la salida de este entrenamiento en el entregable final, ya que vamos a hacer el entrenamiento más informativo en el siguiente punto.

In [23]:
vocabulary_size=len(palabras_unicas)
model=Sequential()
model.add(LSTM(128, input_shape=(SEQ_LENGTH,32)))
model.add(Dense(vocabulary_size,activation='softmax'))
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 128)               82432     
                                                                 
 dense (Dense)               (None, 8774)              1131846   
                                                                 
Total params: 1214278 (4.63 MB)
Trainable params: 1214278 (4.63 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [25]:
optimizer='adam'
model.compile(loss='categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
history=model.fit(X,y, validation_split=0.05, batch_size=128, epochs=5,shuffle=True)

Epoch 1/5
381/381 [==============================] - 7s 13ms/step - loss: 6.7875 - accuracy: 0.0689 - val_loss: 6.9203 - val_accuracy: 0.0730
Epoch 2/5
381/381 [==============================] - 4s 11ms/step - loss: 6.4131 - accuracy: 0.0754 - val_loss: 6.8449 - val_accuracy: 0.0843
Epoch 3/5
381/381 [==============================] - 4s 10ms/step - loss: 6.1664 - accuracy: 0.0839 - val_loss: 6.8526 - val_accuracy: 0.0804
Epoch 4/5
381/381 [==============================] - 4s 10ms/step - loss: 5.9722 - accuracy: 0.0883 - val_loss: 6.8499 - val_accuracy: 0.0773
Epoch 5/5
381/381 [==============================] - 4s 11ms/step - loss: 5.8013 - accuracy: 0.0905 - val_loss: 6.8887 - val_accuracy: 0.0765


In [26]:
probs = model.predict(X)

1601/1601 [==============================] - 3s 2ms/step


In [27]:
print(probs[1])

[2.4221212e-04 1.0867815e-05 7.7835946e-05 ... 9.4469455e-05 4.6561039e-05
 4.0656250e-06]


Para ver cómo evoluciona nuestro modelo del lenguaje, vamos a generar texto según va entrenando. Para ello, vamos a programar una función que, utilizando el modelo en su estado actual, genere texto, con la idea de ver cómo se va generando texto al entrenar cada epoch.

En el código de abajo podemos ver una función auxiliar para obtener valores de una distribución multinomial. Esta función se usará para muestrear el siguiente carácter a utilizar según las probabilidades de la salida de softmax (en vez de tomar directamente el valor con la máxima probabilidad, obtenemos un valor aleatorio según la distribución de probabilidad dada por softmax, de modo que nuestros resultados serán más diversos, pero seguirán teniendo "sentido" ya que el modelo tenderá a seleccionar valores con más probabilidad).



In [ ]:
def sample(probs, temperature=1.0):

  """Nos da el índice del elemento a elegir según la distribución
  de probabilidad dada por probs.
  Args:
  probs es la salida dada por una capa softmax:
  probs = model.predict(x_to_predict)[0]
  temperature es un parámetro que nos permite obtener mayor
  "diversidad" a la hora de obtener resultados.
  temperature = 1 nos da la distribución normal de softmax
  0 < temperature < 1 hace que el sampling sea más conservador,
  de modo que sampleamos cosas de las que estamos más seguros
  temperature > 1 hace que los samplings sean más atrevidos,
  eligiendo en más ocasiones clases con baja probabilidad.
  Con esto, tenemos mayor diversidad pero se cometen más
  errores.
  """

  # Cast a float64 por motivos numéricos
  probs = np.asarray(probs).astype('float64')
  # Hacemos logaritmo de probabilidades y aplicamos reducción
  # por temperatura.
  probs = np.log(probs) / temperature
  # Volvemos a aplicar exponencial y normalizamos de nuevo
  exp_probs = np.exp(probs)
  probs = exp_probs / np.sum(exp_probs)
  # Hacemos el sampling dadas las nuevas probabilidades
  # de salida (ver doc. de np.random.multinomial)
  samples = np.random.multinomial(1, probs, 1)
  return np.argmax(samples)

Utilizando la función anterior y el modelo entrenado, vamos a añadir un callback a nuestro modelo para que, según vaya entrenando, veamos los valores que resultan de generar textos con distintas temperaturas al acabar cada epoch.

Para ello, abajo tenéis disponible el callback *on_epoch_end*. Esta función elige una secuencia de texto al azar en el texto disponible en la variable
text y genera textos de longitud *GENERATED_TEXT_LENGTH* según las temperaturas en *TEMPERATURES_TO_TRY*, utilizando para ello la función *generate_text*.

Completa la función *generate_text* de modo que utilicemos el modelo y la función sample para generar texto.

NOTA: Cuando hagas model.predict, es aconsejable usar verbose=0 como argumento para evitar que la función imprima valores de salida.

In [ ]:
TEMPERATURES_TO_TRY = [0.2, 0.5, 1.0, 1.2]
GENERATED_TEXT_LENGTH = 300
def generate_text(seed_text, model, length, temperature=1):
  """Genera una secuencia de texto a partir de seed_text utilizando model.
  La secuencia tiene longitud length y el sampling se hace con la temperature
  definida.
  """
  # Aquí guardaremos nuestro texto generado, que incluirá el
  # texto origen
  generated = seed_text
  # Utilizar el modelo en un bucle de manera que generemos
  # carácter a carácter. Habrá que construir los valores de
  # X_pred de manera similar a como hemos hecho arriba, salvo que
  # aquí sólo se necesita una oración
  # Nótese que el x que utilicemos tiene que irse actualizando con
  # los caracteres que se van generando. La secuencia de entrada al
  # modelo tiene que ser una secuencia de tamaño SEQ_LENGTH que
  # incluya el último caracter predicho.
  ### TU CÓDIGO AQUÍ
  ### FIN DE TU CÓDIGO
  return generated

In [ ]:
def on_epoch_end(epoch, logs):
  print("\n\n\n")
  # Primero, seleccionamos una secuencia al azar para empezar a predecir
  # a partir de ella
  start_pos = random.randint(0, len(text) - SEQ_LENGTH - 1)
  seed_text = text[start_pos:start_pos + SEQ_LENGTH]
  for temperature in TEMPERATURES_TO_TRY:
    print("------> Epoch: {} - Generando texto con temperature {}".format(epoch + 1, temperature))
    generated_text = generate_text(seed_text, model,
    GENERATED_TEXT_LENGTH, temperature)
    print("Seed: {}".format(seed_text))
    print("Texto generado: {}".format(generated_text))
    print()

generation_callback = LambdaCallback(on_epoch_end=on_epoch_end)

Entrena ahora tu modelo. No te olvides de añadir *generation_callback* a la lista de callbacks utilizados en fit(). Ya que las métricas de clasificación no son tan críticas aquí (no nos importa tanto acertar el carácter exacto, sino obtener una distribución de probabilidad adecuada), no es necesario monitorizar la accuracy ni usar validation data, si bien puedes añadirlos para asegurarte de que todo está en orden.


## Entregable

Completa los apartados anteriores para entrenar modelos del lenguaje que sean capaces de generar texto con cierto sentido. Comentar los resultados obtenidos y cómo el modelo va mejorando época a época. Comentar las diferencias apreciadas al utilizar diferentes valores de temperatura. Entregar al menos la salida de un entrenamiento completo con los textos generados época a época.

El objetivo no es conseguir generar pasajes literarios con coherencia, sino obtener lenguaje que se asemeje en cierta manera a lo visto en el texto original y donde las palabras sean reconocibles como construcciones en castellano. Como ejemplo de lo que se puede conseguir, este es el resultado de generar texto después de 10 epochs y con temperature 0.2 usando El Quijote:


```
-----> Epoch: 10 - Generando texto con temperature 0.2
Seed: o le cautivaron y rindieron el
Texto generado: o le cautivaron y rindieron el caballero de la caballería de la mano de la caballería del cual se le dijo:

-¿quién es el verdad de la caballería de la caballería de la caballería de la caballería de la caballería, y me ha de habían de la mano que el caballero de la mano de la caballería. y que no se le habían de la mano de la c

```

Asimismo, se proponen los siguientes aspectos opcionales para conseguir nota extra:

*   Experimentar con los textos de teatro en verso de Calderón de la Barca (¿es capaz el modelo de aprender las estructuras del teatro en verso?) o con alguno de los otros textos disponibles. También se puede probar con textos de vuestra elección.
*   Experimentar con distintos valores de SEQ_LENGTH.
*   Experimentar con los hiperparámetros del modelo o probar otro tipo de modelos como GRUs o *stacked* RNNs (RNNs apiladas).
*   Experimentar utilizando embeddings en vez de representaciones one-hot.
*   (Difícil) Entrenar un modelo secuencia a secuencia en vez de secuencia a carácter.


